### Importações

In [ ]:
import pandas as pd
import numpy as np
import random


## Constução da matriz de custos CTif

### Acessar a tabela de precificação das transportadoras


In [ ]:

# ---------------Leitura do arquivo de texto da tabela de precificação-----------------

TbPrecificacao = pd.read_csv('TabelaPrecificacao.txt', encoding='ISO-8859-1', sep='\t')
#print(TbPrecificacao.head())

# --------------------Formatação dos valores-------------------------------

# Função para aplicar as transformações em cada valor
def transformar_valor(valor):

    if isinstance(valor, str):
        
        if 'R$' in valor and '-' in valor:
            valor = float(0)
            
        elif 'R$' in valor:
            valor = valor.replace('R$', '').replace(',', '.').strip()
            valor = float(valor)
            
        elif '%' in valor:
            valor = valor.replace('%', '').replace(',', '.').strip()
            valor = float(valor) / 100
    return valor

TbPrecificacaoZero = TbPrecificacao.fillna(0)

# Aplicar a função em todas as células do DataFrame
TbPrecificacaoF = TbPrecificacaoZero.applymap(transformar_valor)

### Acessar e formatar a tabela de notas fiscais separadas por classe


In [ ]:
# Leitura de um arquivo de texto da relacao de notas Classe 1
TbNotasClasse1 = pd.read_csv('Relacao de Transportes i62 - A_H3.txt', encoding='ISO-8859-1', sep='\t', header=None, skiprows=1)


TbNotasFiscais = TbNotasClasse1.iloc[:, [0,1,6,10,11, 12]]
TbNotasFiscais = pd.DataFrame(TbNotasFiscais)

TbNotasFiscais = TbNotasFiscais.rename(columns={
    0: 'NotaFiscal',
    1: 'DataEmissao',
    6: 'TranspRedespacho',
    10: 'Peso',
    11: 'Valor',
    12: 'CustoEmpresa'
})


#print(TbNotasFiscais.head())

# ---------------Formatando a data-----------------------
TbNotasFiscais['DataEmissao'] = pd.to_datetime(TbNotasFiscais['DataEmissao'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')


#print(TbNotasFiscais.head())

### Atribuição de valores aos parâmetros usados no cálculo do custo

In [ ]:

wi = np.array(TbNotasFiscais['Peso'].tolist()) #Peso
#print(wi)

vi = np.array(TbNotasFiscais['Valor'].tolist()) #Valor
#print(vi)



TarifaFrete = np.array(TbPrecificacaoF['TarifaFrete'].tolist())
#print(TarifaFrete)

frf = np.array([0 if x >= 1 else x for x in TarifaFrete]) #_TarifaVariavel
#print(frf_TarifaVariavel)

frff = np.array([0 if x < 1 else x for x in TarifaFrete]) #_TarifaFixa
#print(frff_TarifaFixa)


frminf = np.array(TbPrecificacaoF['FreteMinimo'].tolist()) #_FreteMinimo
#print(frminf_FreteMinimo)

despf = np.array(TbPrecificacaoF['Despacho'].tolist()) #_Despacho
#print(despf_Despacho)


AdValorem = np.array(TbPrecificacaoF['AdValorem'].tolist())
#print(AdValorem)

advf = np.array([0 if x >= 1 else x for x in AdValorem]) #_AdValorem
#print(advf_AdValorem)

advff = np.array([0 if x < 1 else x for x in AdValorem]) #_AdValoremFixo
#print(advff_AdValoremFixo)


grf = np.array(TbPrecificacaoF['Gris'].tolist()) #_GRIS
#print(grf_GRIS)


Pedagio = np.array(TbPrecificacaoF['Pedágio'].tolist())
#print(Pedagio)

pdgf = np.array([0 if x >= 1 else x for x in Pedagio]) #_Pedagio
#print(pdgf_Pedagio)

pdgff = np.array([0 if x < 1 else x for x in Pedagio]) #_PedagioFixo
#print(pdgff_PedagioFixo)

chtf = np.array(TbPrecificacaoF['Canhoto'].tolist()) #_Canhoto
#print(chtf_Canhoto)

ctff = np.array(TbPrecificacaoF['CustoGenerico'].tolist()) #_CustoGenerico
#print(ctff_CustoGenerico)

ICMS = np.array(TbPrecificacaoF['ICMS'].tolist())
#print(ICMS)

WLs = np.array(TbPrecificacaoF['PesoInf'].tolist()) #_PesoMinimo
#print(WLs_PesoMinimo)

WUs = np.array(TbPrecificacaoF['PesoSub'].tolist()) #_PesoMaximo
#print(WUs_PesoMaximo)

### Cálculo do custo por nota fiscal em cada subfaixa de preço

In [ ]:

leni = TbNotasFiscais.shape[0]
lens = TbPrecificacaoF.shape[0]

MtCustoTotal = np.zeros((lens, leni))

for i in range(leni):
    #print('nota', i)
    
    for s in range(lens):
        #print('faixa', s)

        if  frf[s] * wi[i] + frff[s] <= frminf[s]:
            CAFif = frminf[s]

        else: 
            CAFif = (frf[s] * wi[i] + frff[s]) + despf[s] + (advf[s] * vi[i] + advff[s])

        #print(CAFif)

        CDFif = grf[s] * vi[i] + (pdgf[s] * wi[i] + pdgf[s]) + chtf[s] + ctff[s]
        #print(CDFif)
        
        DTif = (CAFif + CDFif)/(1 - ICMS[s])        
        #print(DTif)
        
        MtCustoTotal[s, i] = DTif

subFaixas = len(MtCustoTotal)




### Unificação da lista de valores para as faixas 9, 12, 13 e 14, que contém subfaixas 

In [ ]:

#------------------- Criação das listas das faixas ------------------------------------

subFaixaNove = MtCustoTotal[8]
subFaixaDez = MtCustoTotal[9]
subFaixaOnze = MtCustoTotal[10]
#print(subFaixaNove)

FaixaNove = []

for i in range(leni):
    
    w = wi[i]
    #print(w)
    
    if w <= 51:
        FaixaNove.append(subFaixaNove[i])

    elif 51 <= w <= 100:
        FaixaNove.append(subFaixaDez[i])
    
    elif w >= 101:
        FaixaNove.append(subFaixaOnze[i])

#print(FaixaNove)


subFaixaDoze = MtCustoTotal[11]
subFaixaTreze = MtCustoTotal[12]
subFaixaCatorze = MtCustoTotal[13]

FaixaDoze = []

for i in range(leni):
    
    w = wi[i]
    #print(w)
    
    if w <= 51:
        FaixaDoze.append(subFaixaDoze[i])

    elif 51 <= w <= 100:
        FaixaDoze.append(subFaixaTreze[i])
    
    elif w >= 101:
        FaixaDoze.append(subFaixaCatorze[i])

#print(FaixaDoze)


subFaixaDezessete = MtCustoTotal[16]
subFaixaDezoito = MtCustoTotal[17]
subFaixaDezenove = MtCustoTotal[18]

FaixaTreze = []

for i in range(leni):
    
    w = wi[i]
    #print(w)
    
    if w <= 51:
        FaixaTreze.append(subFaixaDezessete[i])

    elif 51 <= w <= 100:
        FaixaTreze.append(subFaixaDezoito[i])
    
    elif w >= 101:
        FaixaTreze.append(subFaixaDezenove[i])

#print(FaixaTreze)


subFaixaVinte = MtCustoTotal[19]
subFaixaVinteUm = MtCustoTotal[20]
subFaixaVintDois = MtCustoTotal[21]

FaixaCatorze = []

for i in range(leni):
    
    w = wi[i]
    #print(w)
    
    if w <= 51:
        FaixaCatorze.append(subFaixaDezessete[i])

    elif 51 <= w <= 100:
        FaixaCatorze.append(subFaixaDezoito[i])
    
    elif w >= 101:
        FaixaCatorze.append(subFaixaDezenove[i])

#print(FaixaCatorze)

#---------------------------- Substituindo as listas -------------------------------------------

MtCustoFaixa = MtCustoTotal

MtCustoFaixa[8] = FaixaNove
MtCustoFaixa[13] = FaixaDoze
MtCustoFaixa[16] = FaixaTreze
MtCustoFaixa[19] = FaixaCatorze


indices_remover = [9, 10, 14, 15, 17, 18, 20, 21]


### Geração final da matriz MtCustoFaixa

In [ ]:

# -------------------------------Excluindo as listas------------------------------------------------------
MtCustoFaixa = np.delete(MtCustoFaixa, indices_remover, axis=0)

Faixas = len(MtCustoFaixa)
#print(Faixas)
#print(MtCustoTotal)

Notas = MtCustoFaixa.shape
print(Notas)

#print(MtCustoFaixa)

## Criação da lista de notas fiscais agrupadas por destino

In [ ]:
# -------------- Criação da lista para o parâmetro Ij[j]----------------------------

NotasPorDestino = []

for nome, grupo in TbNotasFiscais.groupby('TranspRedespacho'):
    indices = list(grupo.index)
    NotasPorDestino.append(indices)
    
#print(NotasPorDestino)


## Geração do conjunto aleatório de veículos disponíveis


In [ ]:
# --------- Geração do conjunto aleatório de veículos disponíveis -------------

resto = 25
LQuantidadeVeiculo = []
listaConsecutivos = []
ultimo = 1
backultimo = 1

for i in range(8):
    #print('i = ', i)
    #print('ultimo no começo', ultimo)
    
    numeroAleatorio = random.randint(0, resto)
    #print('aleatorio', numeroAleatorio)
    
    resto = resto - numeroAleatorio
   # print('resto', resto)
    
    for a in range(numeroAleatorio):
        
     #   print('ultimo antes do append', ultimo)
        listaConsecutivos.append(ultimo)
        ultimo = ultimo + 1
    
    #print('Lista consecutivos', listaConsecutivos)
    if len(listaConsecutivos) == 0:
        ultimo = backultimo
   #     print('back')
    
    else:
        ultimo = listaConsecutivos[numeroAleatorio - 1]
        
    #print('ultimo numero', ultimo)  
    
    if ultimo != 1:

        LQuantidadeVeiculo.append(listaConsecutivos)
        #print('Bl=', LQuantidadeVeiculo)
        listaConsecutivos = []
        backultimo = ultimo

        ultimo = ultimo + 1
    
    else:
        ultimo = 1

#print(len(LQuantidadeVeiculo))
#print('------------------------------------------')
#print('Bl completa =', LQuantidadeVeiculo)

BlShuffle = LQuantidadeVeiculo.copy()
random.shuffle(BlShuffle)

#print('------------------------------------------')
#print('Bl shuffle =', BlShuffle)

## Atribuição de valor aos parâmetros usados no modelo

In [ ]:

wi = np.array(TbNotasFiscais['Peso'].tolist()) #Peso
#print(wi)

WLs = np.array(TbPrecificacaoF['PesoInf'].tolist()) #_PesoMinimo
#print(WLs_PesoMinimo)

WUs = np.array(TbPrecificacaoF['PesoSub'].tolist()) #_PesoMaximo
#print(WUs_PesoMaximo)


CTif   = MtCustoFaixa.transpose()
#Notas = CTif.shape
#print(Notas)

n = len(TbNotasFiscais)

CTmaxi = []

for i in range(n):
    maxi = np.max(CTif[i, :])
    ctmaxi = 2*maxi
    CTmaxi.append(ctmaxi)


CPredj = 36000 #capacidade de processamento da transportadora de redespacho j em Kg/dia

CPCol  = 90000 #capacidade de processamento da coleta em Kg/dia

CGWc   = [14000, 25000, 27000, 32000, 14000, 24000, 14000, 24000] #peso máximo suportado pelo veículo c (capacidade de transporte) em Kg

OC     = 0.9 #fator de ocupação estipulado como mínimo aos veículos, dado percentualmente

ELf    = TbPrecificacaoF['EntregasInf'].tolist() #quantidade mínima de entregas permitidas ao se escolher a faixa de custo f

EUf    = TbPrecificacaoF['EntregasSup'].tolist() #quantidade máxima de entregas permitidas ao se escolher a faixa de custo f


## Declaração do modelo


In [ ]:
import pyomo.environ as pyo
from pyomo.environ import Set
from pyomo.environ import *
from pyomo.opt import SolverFactory


# Declaração do modelo:

modelo = pyo.ConcreteModel()

# Tamanho dos conjuntos

n = len(TbNotasFiscais)
m = 3
B = 25
G = len(MtCustoFaixa)
H = len(MtCustoTotal)
p = 1


# Índices:

modelo.I = pyo.RangeSet(n) #conjunto I que contem n notas fiscais

modelo.J = pyo.RangeSet(m) #conjunto J que contem m transportadoras

modelo.C = pyo.RangeSet(B) #conjunto C que contem B veículos

modelo.F = pyo.RangeSet(G) #conjunto F que contem G faixas de custo

modelo.S = pyo.RangeSet(H) #conjunto S que contem H subfaixas de custo

modelo.T = pyo.RangeSet(p) #conjunto T que contem p períodos de tempo

Fc = [[1,2],[3],[4],[5,6],[7,8,9],[10,11,12],[13],[14]]

Ij = NotasPorDestino



### Declaração das variáveis de decisão do modelo

In [ ]:
# --------------------------- Variáveis de decisão --------------------------------------------------------------

modelo.xifct = pyo.Var(modelo.I, modelo.F, modelo.C, modelo.T, within=pyo.Binary)

modelo.ufct  = pyo.Var(modelo.F, modelo.C, modelo.T, within=pyo.Binary)

modelo.zjfct = pyo.Var(modelo.J, modelo.F, modelo.C, modelo.T, within=pyo.Binary)

modelo.si    = pyo.Var(modelo.I, within=pyo.Binary) #within=pyo.NonNegativeReals)

### Declaração da função objetivo

In [ ]:
# ---------------------------- Função Objetivo -------------------------------------------------------------------

modelo.objetivo = pyo.Objective(
    expr = 
        sum(sum(sum(sum(sum(CTif[i-1][f-1]*modelo.xifct[i,f,c,t]
        for t in modelo.T)
        for f in Fc[l])
        for c in BlShuffle[l])
        for l in range(len(BlShuffle)))
        for i in modelo.I)
    
        + sum(CTmaxi[i-1]*modelo.si[i]
        for i in modelo.I), 
    sense = pyo.minimize)

#resultado = modelo.objetivo()
#print(resultado)


### Declaração das restrições do modelo

In [ ]:
# ----------------- Despacho único de cada carga ----------- (8) -------------------------
modelo.RestrDespachoCarga = pyo.ConstraintList()


for i in modelo.I:
    modelo.RestrDespachoCarga.add(expr = 
        sum(sum(sum(sum(modelo.xifct[i,f,c,t] + modelo.si[i] 
            for f in Fc[l]) for c in BlShuffle[l]) for l in range(len(BlShuffle))) for t in modelo.T) == 1)

In [ ]:
# ----------------- Atribuição única de faixa de custo ----------- (9) -------------------------

modelo.RestrFaixaUnica = pyo.ConstraintList()

for l in range(len(BlShuffle)):
    for c in BlShuffle[l]:      
        for t in modelo.T:
            modelo.RestrFaixaUnica.add(expr =
                sum(modelo.ufct[f, c, t]
                    for f in Fc[l]) <= 1)   

In [ ]:

# ----------------- Capacidade de processamento (90 ton) ----------- (10) -------------------------

modelo.RestrProcessamento = pyo.ConstraintList()

for i in modelo.I:
    modelo.RestrProcessamento.add(expr = 
        sum(sum(sum(sum(modelo.xifct[i,f,c,t] * wi[i-1] 
            for f in Fc[l]) for c in BlShuffle[l]) for l in range(len(BlShuffle))) for t in modelo.T) <= CPCol)

In [ ]:
# ----------------- Capacidade de coleta (36 ton) ----------- (11) -------------------------

modelo.RestrColeta = pyo.ConstraintList()

for j in modelo.J:
    for i in Ij[j-1]:
        modelo.RestrColeta.add(expr = 
            sum(sum(sum(sum(modelo.xifct[i+1,f,c,t] * wi[i+1] 
                for f in Fc[l]) for c in BlShuffle[l]) for l in range(len(BlShuffle))) for t in modelo.T) <= CPredj)

In [ ]:
# ----------------- Capacidade do veículo ----------- (12) -------------------------      

modelo.RestrCapacidadeVeiculo = pyo.ConstraintList()

for l in range(len(BlShuffle)):
    for c in BlShuffle[l]:
        for f in Fc[l]:
            for t in modelo.T:
                modelo.RestrCapacidadeVeiculo.add(expr = modelo.ufct[f, c, t] * OC * CGWc[l] <=                                                          
                                                        sum(modelo.xifct[i,f,c,t] * wi[i-1] for i in modelo.I)) 

                modelo.RestrCapacidadeVeiculo.add(expr = sum(modelo.xifct[i,f,c,t] * wi[i-1] for i in modelo.I) <=
                                                        modelo.ufct[f, c, t] * CGWc[l])

In [ ]:
# ----------------- Linearização ----------- (13) -------------------------   

modelo.RestrLinearizacao13 = pyo.ConstraintList()

for j in modelo.J:
    for i in Ij[j-1]:
        for l in range(len(BlShuffle)):
            for c in BlShuffle[l]:
                for f in Fc[l]:
                    for t in modelo.T:
                        modelo.RestrLinearizacao13.add(expr =
                            modelo.xifct[i+1,f,c,t] <= modelo.zjfct[j,f,c,t])


In [ ]:
# ----------------- Linearização ----------- (14) ------------------------- 

modelo.RestrLinearizacao14 = pyo.ConstraintList()

for j in modelo.J:
        for l in range(len(BlShuffle)):
            for c in BlShuffle[l]:
                for f in Fc[l]:
                    for t in modelo.T:
                        modelo.RestrLinearizacao14.add(expr =
                            modelo.zjfct[j,f,c,t] <= sum(modelo.xifct[i+1,f,c,t] for i in Ij[j-1]))

In [ ]:
# ----------------- Número de entregas x faixa ----------- (15) ------------------------- 

modelo.RestrIntervaloEntrega = pyo.ConstraintList()

for l in range(len(BlShuffle)):
        for c in BlShuffle[l]:
            for f in Fc[l]:
                modelo.RestrIntervaloEntrega.add(expr = modelo.ufct[f, c, t] * ELf[f] <= 
                                                 sum(sum(modelo.zjfct[j,f,c,t] for j in modelo.J) for t in modelo.T)) 

                modelo.RestrIntervaloEntrega.add(expr = sum(sum(modelo.zjfct[j,f,c,t] for j in modelo.J) 
                                                            for t in modelo.T) <= EUf[f])

        

In [ ]:
 # ----------------- Evitar simetria de soluções ----------- (16) ------------------------- 
    
modelo.RestrSimetria = pyo.ConstraintList()

for i in modelo.I:
    for l in range(len(BlShuffle)):
        for c in BlShuffle[l]:
            for f in Fc[l]:
                for t in modelo.T:
                    if 2 <= t <=p:
                        modelo.RestrSimetria.add(expr =
                        pyo.quicksum(modelo.xifct[i,f,c,t-1] * wi[i-1]) >= pyo.quicksum(modelo.xifct[i,f,c,t] * wi[i-1])) 

                    else:
                        pass

## Solução do modelo

In [ ]:
# Criar solver GLPK
solver = SolverFactory('glpk')
solver.options['tmlim'] = 180

# Resolver o modelo utilizando GLPK
results = solver.solve(modelo)

print(f"Valor da função objetivo: {modelo.objetivo():.2f}")
print(results)
# Imprimir os valores das variáveis e o valor da função objetivo

# Imprimir os valores das variáveis e o valor da função objetivo
print(f"Status da otimização: {results.solver.status}")
print(f"Valor da função objetivo: {modelo.objetivo():.2f}")

# Imprimir valores da variável xifct
for i in modelo.I:
    for l in range(len(BlShuffle)):
        for c in BlShuffle[l]:
            for f in Fc[l]:  
                for t in modelo.T:
                    print(f'x[ {i} {f} {c} {t} ] = {pyo.value(modelo.xifct[i,f,c,t])}') 
                pass

# Imprimir valores da variável si
for i in modelo.I:
    print(f's[ {i} ] = {pyo.value(modelo.si[i])}')
    pass

        
# Imprimir valores da variável ufct
for l in range(len(BlShuffle)):
    for c in BlShuffle[l]:
        for f in Fc[l]:  
            for t in modelo.T:
                print(f'u[ {f} {c} {t} ] = {pyo.value(modelo.ufct[f,c,t])}') 
            pass

# Imprimir valores da variável zjfct
for j in modelo.J:
    for l in range(len(BlShuffle)):
        for c in BlShuffle[l]:
            for f in Fc[l]:  
                for t in modelo.T:
                    print(f'z[ {j} {f} {c} {t} ] = {pyo.value(modelo.zjfct[j,f,c,t])}') 
                pass
    

